# 1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars. 

In [1]:
#importing neccessary library's
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,WebDriverException,ElementNotInteractableException,TimeoutException
import urllib.request
from selenium.webdriver.common.keys import Keys
import re
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests as rqt
import shutil

In [9]:
#loading webpage
driver=webdriver.Edge()
driver.get("http://www.amazon.in/")
driver.maximize_window()
try:
    while True:
        search_product_name=input("Enter the product name: ")
        if len(search_product_name)==0:
            print("You haven't enterd the product!!!!")
        elif len(search_product_name)>0:
            break
            
    search_box=driver.find_element(By.XPATH,'//input[@id="twotabsearchtextbox"]')
    search_box.send_keys(search_product_name)
    driver.find_element(By.XPATH,'//input[@id="nav-search-submit-button"]').click()
except NoSuchElementException as e:
    print(e)
    

Enter the product name: guitar


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. 
In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. 
Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [10]:
#declaring necessary lists
brand=[]
productName=[]
price=[]
returnExchange=[]
expectedDelivery=[]
Availability=[]
url_list=[]

#loading url
try:
    for page in range(3):
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
        url=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        for i in url:
            url_list.append(i.get_attribute('href'))
        nextTag= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')))
        nextTag.click()
        driver.implicitly_wait(10)
except:
    pass
    

#iterating through each link to get product details
for link in url_list:
    driver.get(link)
    driver.implicitly_wait(2)
    try:
        #loading brand name
        brandTag=driver.find_element(By.XPATH,'//div[@id="detailBullets_feature_div"]/ul/li[3]/span[1]/span[2]')
        brand.append(brandTag.text)
    except NoSuchElementException as e:
        try:
            brandTag=driver.find_element(By.XPATH,'//table[@class="a-normal a-spacing-micro"]/tbody/tr[1]/td[2]/span')
            brand.append(brandTag.text)
        except NoSuchElementException as e:
            brand.append('-')
            
    try:
        #loading product name
        productNameTag=driver.find_element(By.XPATH,'//div[@id="titleSection"]/h1/span')
        productName.append(productNameTag.text)
    except NoSuchElementException as e:
        productName.append('-')
    try:
        #loading product price
        priceTag=driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none aok-align-center"]/span/span[2]')
        if (priceTag.text is None):
            price.append('-')
        else:
            price.append(priceTag.text)
    except NoSuchElementException as e:
        try:
            priceTag=driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[5]/div[3]/div[4]/div[12]/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/span[1]/span[2]')
            price.append(priceTag.text)
        except NoSuchElementException as e:
            price.append('-')
    try:
        #loading return and exchange  policy
        returnTag=driver.find_element(By.XPATH,'//div[@id="RETURNS_POLICY"]')
        returnExchange.append(returnTag.text)
    except NoSuchElementException as e:
        returnExchange.append('-')
    try:
        #loading expected delivery date
        deliveryTag=driver.find_element(By.XPATH,'//div[@id="mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE"]/span/span')
        expectedDelivery.append(deliveryTag.text)
    except NoSuchElementException as e:
        expectedDelivery.append('-')
    try:
        #loading avalability status
        avalTag=driver.find_element(By.XPATH,'//div[@id="availability"]/span')
        Availability.append(avalTag.text)
    except NoSuchElementException as e:
        Availability.append('-')
    

In [11]:
#creating dataframe and sving into csv file
df=pd.DataFrame({"Brand":brand,"Product Name":productName,"Price":price,"Return?Exchange Policy":returnExchange,"Expected Delivery Date":expectedDelivery,"Avalibality":Availability,"Link":url_list})
df.to_csv('AmazonShirtsData.csv')
df

,Brand,Product Name,Price,Return?Exchange Policy,Expected Delivery Date,Avalibality,Link
0,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","₹4,999",7 days Replacement,"Friday, 8 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Medellin,Medellin Acoustic Guitar (with guitar learning...,"₹2,499",7 days Replacement,"Saturday, 9 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Intern,"Intern INT-38C Right hand Acoustic Guitar Kit,...","₹1,899",7 days Replacement,"Wednesday, 6 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Kadence,Kadence Frontier guitar with Online Guitar lea...,"₹4,999",7 days Replacement,"Wednesday, 6 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹1,999",7 days Replacement,"Wednesday, 6 September",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
80,JUAREZ,JUAREZ Arpéggio 104.14 cm (41 Inch) Acoustic G...,"₹6,059",7 days Replacement,"Wednesday, 6 September",In stock,https://www.amazon.in/JUAREZ-Arp%C3%A9ggio-Aco...
81,blueberry,"Blueberry B40L, 40"" Acoustic Guitar Kit Inbuil...","₹4,290",7 days Replacement,"Saturday, 9 September",In stock,https://www.amazon.in/Blueberry-Acoustic-Inbui...
82,Poonam_Enterprise,Poonam_Enterprise Plastic Musical Guitar (Yellow),₹999,7 days Replacement,"Thursday, 7 September",Only 2 left in stock,https://www.amazon.in/Poonam_Enterprise-Yellow...
83,blueberry,"Blueberry, VIP-340, 34Inch Acoustic Guitar Kit...","₹2,999",7 days Replacement,"Saturday, 9 September",In stock,https://www.amazon.in/Blueberry-VIP-340-34Inch...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [18]:
#getting driver
driver=webdriver.Edge()
imagesList=['fruits' ,'cars' ,'Machine Learning' ,'Guitar' ,'Cakes']
image_src=[]
#searching item by iterarting in list
for i in range(5):
    #loading webpage
    driver.get('https://images.google.com/')
    #screen zoom
    driver.maximize_window()
    #search button an dkey sending
    driver.find_element(By.NAME,'q').send_keys(imagesList[i])
    
    #submitting
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME,"Tg7LZd"))).click()
    time.sleep(2)
    #img tag searching
    try:
        im=driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
        #Iterating on tag and loading src of image
        for img in im:
            src=img.get_attribute('src')
            if src is not None:
                if(src[0:4]=='http'):
                     image_src.append(src)
    except NoSuchElementException as e:
        pass
        

#print(image_src)
for i in range(len(imagesList)):
    for j in range(len(image_src)):
        if j > 10:
            break
        src_load=rqt.get(image_src[j])
        path=r"D:\internship\img\{}".format(imagesList[i])+str(j)+".jpg"
        file=open(path,"wb")
        file.write(src_load.content)
        


['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTR4RnPU4rUtrOzKuxFoZQ93yDPYFFGWsDQ8Q&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5AhK7vhCeytMJ9y8tepsRj7t595WYUY2mBg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRG7CtEZMZn6PUABnYajBy6Aq8Hpinus_BVeg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmu9OJRAE3HE04HzrITLVXCHN3iL2tTJgZ1g&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRxCG9GHb8xvSNg6sZ6Gk6KJ4HMxR8ICfNqDg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRnzJSk3TAwQ3DsAFSCOc7Je5m9_Fz53wVS4Q&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQiadds9ARDkjHgG2dwUCFFjcPM3n9lb_dssw&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTmNoM0ipUXaVPylq5Hf2ne4C1uKF4SGmuuTA&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQege_7Vs5qYWZ5V2817FgmRwxa5A6xnc4oQw&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIFDu4b2QLQyj4Zpn

# Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page.
Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [10]:
#loading driver
driver=webdriver.Edge()
#loading webpage
driver.get('http://www.flipkart.com/')
driver.maximize_window()
#getting mobile name from user
mobileName=input("Enter the mobile name: ")
#declaring lists to store the info
BrandName=[]
SmartphoneName=[]
Colour=[]
RAM=[]
Storage=[]
PrimaryCamera=[]
SecondaryCamera=[]
DisplaySize=[]
BatteryCapacity=[]
Price=[]
ProductURL=[]
#cancling log in popup
try:
    driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]').click()
except:
    pass

#finding search tag and sending keys by hitting enter button    
try:
    search=driver.find_element(By.NAME,'q')
    search.send_keys(mobileName)
    search.send_keys(Keys.ENTER)
except NoSuchElementException as e:
    print("search box not found",e.args())
#getting mobil urls and adding into list    
try:
    links=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"_1fQZEK")))
    for i in links:
        ProductURL.append(i.get_attribute('href'))
except NoSuchElementException as e:
    print(e)
    ProductURL.append('-')
    
#iterating through each page and extracting information
for link in ProductURL:
    #continue if no link avilabel
    if str(link)=='-':
        continue
    #hitting link
    driver.get(link)
    try:
         #extracting brand name
        brandTag=driver.find_element(By.XPATH,'//div[@class="aMaAEs"]/div/h1/span')
        name=re.findall(r'^[A-za-z]+',str(brandTag.text))
        BrandName.append(re.sub(r'\[|\]','',str(name)).strip())
    except NoSuchElementException as e:
        BrandName.append('-')
    #expanding mobile information
    driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]').click()
    #getting smartphone name
    try:
                                                
        modelTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li')
        name=str(re.sub(r'\[|\]','',str(name))) + ' ' +str( modelTag.text)
        SmartphoneName.append(name)            
    except NoSuchElementException as e:         
        SmartphoneName.append('-')
    #getting color details
    try:
        colorTag=driver.find_element(By.XPATH,'//table[1]/tbody[1]/tr[4]//td[2]/ul[1]/li')
        Colour.append(colorTag.text)
    except NoSuchElementException as e:
        Colour.append('-')
    #extracting ram value    
    try:
        ramTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(ramTag.text)
    except NoSuchElementException as e:
        RAM.append('-')
    #storage information extraction
    try:
        storageTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage.append(storageTag.text)
    except NoSuchElementException as e:
        Storage.append('-')
    #loading primary camera details
    try:
        pCameraTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        PrimaryCamera.append(pCameraTag.text)
    except NoSuchElementException as e:
        PrimaryCamera.append('-')
    #loading secondary camera details  
    try:
        sCameraTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li')
        SecondaryCamera.append(sCameraTag.text)
    except NoSuchElementException as e:
        SecondaryCamera.append('-')
    #loading display size details
    try:
        displayTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        DisplaySize.append(displayTag.text)
    except NoSuchElementException as e:
        DisplaySize.append('-')
    #scraping battery details    
    try:
        batteryTag=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr[1]/td[2]/ul/li')
        BatteryCapacity.append(batteryTag.text)
    except NoSuchElementException as e:
        BatteryCapacity.append('-')
    #scraping price  
    try:
        priceTag=driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(priceTag.text)
    except NoSuchElementException as e:
        Price.append('-')
        
    
    
#creating dataframe
df=pd.DataFrame({"Brand":BrandName,"SmartPhoneName":SmartphoneName,"Color":Colour,"RAM":RAM,"Storage":Storage,"PrimaryCamera":PrimaryCamera,"SecondaryCamera":SecondaryCamera,"DisplaySize":DisplaySize,"BatteryCapacity":BatteryCapacity,"Price":Price,"ProductURL":ProductURL})
df 
    



Enter the mobile name: iphone 13


,Brand,SmartPhoneName,Color,RAM,Storage,PrimaryCamera,SecondaryCamera,DisplaySize,BatteryCapacity,Price,ProductURL
0,'APPLE','APPLE' iPhone 13,Midnight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹59,999",https://www.flipkart.com/apple-iphone-13-midni...
1,'APPLE','APPLE' iPhone 13,Green,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹59,999",https://www.flipkart.com/apple-iphone-13-green...
2,'APPLE','APPLE' iPhone 13,Blue,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹59,999",https://www.flipkart.com/apple-iphone-13-blue-...
3,'APPLE','APPLE' iPhone 13,Pink,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹59,999",https://www.flipkart.com/apple-iphone-13-pink-...
4,'APPLE','APPLE' iPhone 13,Starlight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹59,999",https://www.flipkart.com/apple-iphone-13-starl...
5,'APPLE','APPLE' iPhone 13,(PRODUCT)RED,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹59,999",https://www.flipkart.com/apple-iphone-13-produ...
6,'APPLE','APPLE' iPhone 13,Green,-,256 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹69,999",https://www.flipkart.com/apple-iphone-13-green...
7,'APPLE','APPLE' iPhone 13,Pink,-,256 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹69,999",https://www.flipkart.com/apple-iphone-13-pink-...
8,'APPLE','APPLE' iPhone 13,Starlight,-,256 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),3240 mAh,"₹69,999",https://www.flipkart.com/apple-iphone-13-starl...
9,'APPLE','APPLE' iPhone 13,(PRODUCT)RED,-,512 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),71.5 mm,"₹89,999",https://www.flipkart.com/apple-iphone-13-produ...


# 5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [75]:
#loading webpage
driver=webdriver.Chrome()
driver.get('https://www.google.co.in/maps/@17.4303456,78.3273134,15z?entry=ttu')
driver.maximize_window()
#search tag loading
searchTag=driver.find_element(By.NAME,'q')
searchTag.send_keys(input("Enter city name: "))
searchTag.send_keys(Keys.ENTER)
#
url = driver.current_url
coordinates = url.split("@")[1].split(",")
latitude = coordinates[0]
longitude = coordinates[1]
print("latitude: ",latitude)
print("longitude: ",longitude)

Enter city name: pune
latitude:  17.4303456
longitude:  78.3273134


# 6.Write a program to scrap all the available details of best gaming laptops from 
digit.in

In [78]:
#loading webpage
driver=webdriver.Edge()
driver.get('https://www.digit.in/')
driver.maximize_window()
#going through gaming section

WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[2]/div/ul/li[5]/span'))).click()
WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[2]/div/ul/li[5]/div[2]/div/div[2]/div/ul[2]/li[2]/a'))).click()
#declaring list
links=[]
laptop_Name=[]
desc=[]
os=[]
laptop_display=[]
processor=[]
memory=[]
laptop_price=[]
#extracting url's of diff lapotops list 
for i in range(1,6):
    url=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="categories_listing"]/div[1]/div/div/ul/li['+str(i)+']/a[1]')))
    links.append(url.get_attribute('href'))
#getting all details of gaming laptops 
for link in links:
    driver.get(str(link))
        #loading laptop name
    try:
        lpname=driver.find_elements(By.XPATH,'//table[@id="summtable"]/tbody[1]/tr/td[1]')
        for i in lpname:
            laptop_Name.append(i.text)
    except NoSuchElementException as e:
        laptop_Name.append('-')
        #loading description of laptop
    try:
        laptop_descTag=driver.find_elements(By.CLASS_NAME,'tptn-prod-desc')
        for i in laptop_descTag:
            desc.append(i.text)
    except NoSuchElementException as e:
        desc.append('-')
        #loading OS info
    try:
        laptop_osTag=driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div[1]/ul[1]/li[1]/div[1]/div[1]')
        for i in laptop_osTag:
            os.append(i.text)
    except NoSuchElementException as e:
        os.append('-')
        #loading laptop display size
    try:
        dis=driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div[1]/ul[1]/li[2]/div[1]/div[1]')
        for i in dis:
                laptop_display.append(i.text)
    except NoSuchElementException as e:
        laptop_display.append('-')
        #loading processor type
    try:
        laptop_processorTag=driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div[1]/ul[1]/li[4]/div[1]/div[1]')
        for i in laptop_processorTag:
            processor.append(i.text)
    except NoSuchElementException as e:
        processor.append('-')
        #loading memory limit of laptop
    try:
        laptop_storageTag=driver.find_elements(By.XPATH,'//div[@class="product-detail"]/div[1]/ul[1]/li[5]/div[1]/div[1]')
        for i in laptop_storageTag:
            memory.append(i.text)
    except NoSuchElementException as e:
        memory.append('-')
            #loading price of laptop
    try:
        laptop_priceTag=driver.find_elements(By.XPATH,'//table[@id="summtable"]/tbody[1]/tr/td[3]')
        for i in laptop_priceTag:
            laptop_price.append(i.text)
    except NoSuchElementException as e:
        laptop_price.append('-')
        
driver.quit()



    

In [83]:
#creating dataframe( not using description)
df=pd.DataFrame({"Laptop Name":laptop_Name,"OS":os,"Display Size":laptop_display,"Processor":processor,"Price":laptop_price,"Description":desc})
df=df.replace('NA','-')
df=df.replace('','-')
df

,Laptop Name,OS,Display Size,Processor,Price,Description
0,Lenovo IdeaPad Gaming 3,Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5-5600H | 3.3 GHz,"₹ 54,490",There are two things for which you’d buy a Len...
1,Acer Aspire 7,Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5-5500U | 2.1 GHz,"₹ 49,990",The Acer Aspire 7 is another gaming laptop tha...
2,Lenovo IdeaPad Gaming 3,Windows 11 Home,"15.6"" (1920 x 1080)",11th Gen Intel Core i5-11320H | 3.2 GHz,"₹ 57,890",It is basically the same Lenovo IdeaPad Gaming...
3,HP Pavilion Gaming 15,Windows 10 Home,"15.6"" (1920 x 1080)",5th Gen AMD Ryzen 5 5600H | 4.2 GHz,"₹ 60,490",The HP Pavilion Gaming 15 is on this list than...
4,Acer Aspire 7,Windows 11 Home,"15.6"" (1920 x 1080)",12th Gen Intel Core i5-1240P | 3.30 GHz,"₹ 52,990",For users looking for the ultimate balance bet...
5,Lenovo IdeaPad Gaming 3,Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5-5600H | 3.2 GHz,"₹ 49,990",The size of modern games can easily exceed 100...
6,MSI Bravo 15,Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5-5600H | 3.3 GHz,"₹ 47,990",While most laptops on this list feature an Nvi...
7,OMEN by HP Laptop 17-ck1023TX,Windows 11 Home,"17.3"" (2560 x 1440)",12th Gen Intel Core i9-12900H | 5.0 GHz,"₹ 349,475",Starting things off with the OMEN by HP Laptop...
8,Victus by HP Laptop 16-e1060AX,Windows 11 Home,"16.1"" (1920 x 1080)",AMD Ryzen 7 6800H | 4.7 GHz,"₹ 93,490","Like we mentioned in the intro, Victus is HP's..."
9,HP Pavilion Gaming Laptop 15 ec2145AX,Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5 5600H | 4.2 GHz,"₹ 74,500",The HP gaming laptop Pavilion series is a long...


# 7.Write a python program to scrape the details for all billionaires from www.forbes.com. 
Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [17]:
#loading driver and page
driver=webdriver.Edge()
driver.get('https://www.forbes.com/')
driver.maximize_window()

#navigating through menu
WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"/html[1]/body[1]/div[1]/header[1]/nav[1]/div[1]/div[1]/div[1]/div[1]/*[name()='svg'][1]"))).click()
b=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[1]/a")))
Hover = ActionChains(driver).move_to_element(b)
Hover.perform()
ab=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"/html[1]/body[1]/div[1]/header[1]/nav[1]/div[1]/div[1]/div[1]/div[2]/ul[1]/li[2]/div[2]/div[3]/ul[1]/li[1]/a[1]")))
ab.click()

#declaring lists
Rank=[]
Name=[]
NetWorth=[]
Age=[]
Country=[]
source=[]
Industry=[]

#extracting rank
try:
    rankTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[1]/div[1]')))
    for i in rankTag:
        Rank.append(i.text)
except NoSuchElementException as e:
    Rank.append('-')
    
#extracting name
try:
    nameTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[2]/div[1]')))
    for i in nameTag:
        Name.append(i.text)
except NoSuchElementException as e:
    Name.append('-')
    
#extracting networth
try:
    networthTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[3]/div[1]')))
    for i in networthTag:
        NetWorth.append(i.text)
except NoSuchElementException as e:
    NetWorth.append('-')

    
#extracting age
try:
    ageTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[4]/div[1]')))
    for i in ageTag:
        Age.append(i.text)
except NoSuchElementException as e:
    Age.append('-')
    
    
#extracting country name
try:
    countryTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[5]')))
    for i in countryTag:
        Country.append(i.text)
except NoSuchElementException as e:
    Country.append('-')
    
    
#extracting source details
try:
    sourceTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[6]/div[1]')))
    for i in sourceTag:
        source.append(i.text)
except NoSuchElementException as e:
    source.append('-')
    
    
#extracting source details
try:
    industryTag=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="Table_tableBlock__rRFM8 Table_billionaires__qIfS_"]/div[2]/div[2]/div[2]/div[1]/div[*]/div[1]/div[*]/div[*]/div[1]/div[6]/div[1]')))
    for i in industryTag:
        Industry.append(i.text)
except NoSuchElementException as e:
    Industry.append('-')


In [24]:
#creating dataframe
df=pd.DataFrame({"Rank":Rank,"Name":Name,"NetWorth":NetWorth,"Age":Age,"Country":Country,"Sourcr":source,"Industry":Industry})
df

,Rank,Name,NetWorth,Age,Country,Sourcr,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,LVMH
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX","Tesla, SpaceX"
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Amazon
3,4,Larry Ellison,$107 B,78,United States,Oracle,Oracle
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Berkshire Hathaway
...,...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Solar panels,Solar panels
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,Chemicals
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food","Candy, pet food"
198,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food","Candy, pet food"


# 8.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [2]:
#loading driver and page
try:
    driver=webdriver.Edge()
    driver.get('https://youtu.be/97F8kV086As?si=niB3lypGC5zCIioJ')
    driver.maximize_window()
except WebDriverException as e:
    driver.quit()
    driver=webdriver.Edge()
    driver.get('https://youtu.be/97F8kV086As?si=niB3lypGC5zCIioJ')
    driver.maximize_window()
#declaring list
comment=[]
upvote=[]
ctime=[]
wait=WebDriverWait(driver,20)

for i in range(7):
    #scrolling page to load comments page
    try:
        body=wait.until(EC.visibility_of_element_located((By.TAG_NAME,'body')))
        time.sleep(1)
        body.send_keys(Keys.END)
    except StaleElementReferenceException as e:
        body=wait.until(EC.visibility_of_element_located((By.TAG_NAME,'body')))
        time.sleep(1)
        body.send_keys(Keys.END)    
    time.sleep(10)
#extracting comments
    try:
        try:
            driver.find_element(By.XPATH,'/html/body/ytd-app/ytd-popup-container/tp-yt-paper-dialog/yt-mealbar-promo-renderer/div/div[2]/yt-button-renderer[1]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
        except NoSuchElementException as e:
            pass
        except ElementNotInteractableException as e:
            pass
        
        commentTag=wait.until(EC.visibility_of_all_elements_located((By.ID,"content-text")))
        for i in commentTag:
            comment.append(i.text)
    except NoSuchElementException as e:
        comment.append('-')
        
#extracting likes        
    try:
        try:
            driver.find_element(By.XPATH,'/html/body/ytd-app/ytd-popup-container/tp-yt-paper-dialog/yt-mealbar-promo-renderer/div/div[2]/yt-button-renderer[1]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
        except NoSuchElementException as e:
            pass
        except ElementNotInteractableException as e:
            pass
        time.sleep(5)
        upvoteTag=driver.find_elements(By.ID,"vote-count-middle")
        for i in upvoteTag:
            upvote.append(i.text)
    except NoSuchElementException as e:
        upvote.append('-')
#extracting comment time        
    try:
        try:
            driver.find_element(By.XPATH,'/html/body/ytd-app/ytd-popup-container/tp-yt-paper-dialog/yt-mealbar-promo-renderer/div/div[2]/yt-button-renderer[1]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
        except NoSuchElementException as e:
            pass
        except ElementNotInteractableException as e:
            pass
        ctimeTag=wait.until(EC.visibility_of_all_elements_located((By.XPATH,'//div[@id="header-author"]/yt-formatted-string[1]/a[1]')))
        for i in ctimeTag:
            ctime.append(i.text)
    except NoSuchElementException as e:
        ctime.append('-')
    


In [3]:
#creating dataframe
df=pd.DataFrame({"Comment":comment,"Likes":upvote,"Time":ctime})
df

,Comment,Likes,Time
0,214 Runs 5th Wicket partnership b/w Babar Azam...,231,20 hours ago
1,It was the first time that I did not feel sad ...,177,20 hours ago
2,Iftikhar and his Power Hitting in the end Next...,108,20 hours ago
3,"-Babar took 72 balls to reach fifty, 37 balls ...",159,20 hours ago
4,"Babar - Class & Technique \nIftikhar - ""Chacha...",60,20 hours ago
...,...,...,...
554,We want to see same Babar in Whole Asia cup,1,14 hours ago
555,Well played babar and iftikhar,,16 hours ago
556,Do you agree with me?\nBabar is the best batsm...,4,20 hours ago (edited)
557,Fakhar Zaman what a catch,,10 hours ago


# 9.Write a python program to scrape a data for all available Hostels from 
https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description. 

In [12]:
#loading driver and page
driver=webdriver.Chrome()
driver.get('https://www.hostelworld.com/')
driver.maximize_window()
#declaring list
hostel_links=[]
hostel_name=[]
distance=[]
ratings=[]
totalrev=[]
overallre=[]
pr_from_price=[]
d_from_price=[]
h_facilities=[]
h_desc=[]
#getting search tag
try:
    
    searchTag=driver.find_element(By.XPATH,'//div[@class="input input-strip"]/div[1]/div[2]/input[1]')
    searchTag.send_keys('London')
    optionTag=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[text()='London']"))).click()
    time.sleep(5)
    button=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html[1]/body[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/button[2]/span[1]'))).click()
except NoSuchElementException as e:
    print(e)
while True:
    #getting links of all hostels 
    time.sleep(10)
    hl=WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,'nuxt-link')))
    for i in hl:
        hostel_links.append(i.get_attribute('href'))
    
    #loading distance from city
    try:
        dist=WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,'distance-description')))
        for i in dist:
            distance.append(i.text)
    except NoSuchElementException as e:
        distance.append('-')
        
    
    try:
        nextbutton=WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//div[@class="pagination-wrapper"]/div[1]//button[2]')))
        driver.switch_to.active_element
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
        time.sleep(2)
        nextbutton.click()
    except TimeoutException as e:
        break
#     Hover = ActionChains(driver).move_to_element(nextbutton)
#     Hover.click()
#     Hover.perform()
    
    

#visiting each link    
for link in hostel_links[2:]:
    driver.get(link)
    try:
        hname=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.CLASS_NAME,'title-2')))
        hostel_name.append(hname.text)
    except TimeoutException as e:
        hostel_name.append('-')
        
        
    #loading rating
    try:
        rat=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="initial-property-info"]/div[1]/div[1]/div[1]')))
        ratings.append(rat.text)
    except TimeoutException as e:
        ratings.append('-')
        
    #loading total review
    try:
        tot=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="initial-property-info"]/div[1]/div[1]/div/div[2]')))
        totalrev.append(re.findall(r'\d+',str(tot.text)))
    except TimeoutException as e:
        totalrev.append('-')
        
    #loading over all rating
    try:
        oall=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="initial-property-info"]/div[1]/div[1]/div[2]/div[1]')))
        overallre.append(oall.text)
    except TimeoutException as e:
        overallre.append('-')
        
    try:
        price=(WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="dorms-first"]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[1]')))) or (WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,'//div[@class="dorms-first"]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[1]'))))
        d_from_price.append(price.text)
    except TimeoutException as e:
         d_from_price.append("-")
    try:
        pr=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="dorms-first"]/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[1]')))
        pr_from_price.append(pr.text)
    except TimeoutException as e:
         pr_from_price.append("-")
        
        #loading facilities
    try:
        hf=WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,'//div[@class="facilities-container"]/ul[1]/li[*]/ul/li')))
        temp=[]
        for i in hf:
            temp.append(i.text)
        h_facilities.append(temp)
    except TimeoutException as e:
        h_facilities.append('-')
        
        #loading description
    try:
        hdsc=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,'//div[@class="description-container"]/div[1]/div[2]')))
        h_desc.append(hdsc.text)
    except TimeoutException as e:
        h_desc.append('-')
    
    

In [19]:
#creating dataframe
df=pd.DataFrame({"Name":hostel_name  ,"Distance from city centre":distance ,"Rating":ratings ,"Total reviews":totalrev,"Over all reviews":overallre,"Facilities":h_facilities,"Description":h_desc,"Privates from":pr_from_price,"Dorms from":d_from_price})
df

,Name,Distance from city centre,Rating,Total reviews,Over all reviews,Facilities,Description,Privates from,Dorms from
0,Wombat's City Hostel London,- 3.6km from city centre,9.0,[15279],Superb,"[Linen Included, Free City Maps, Free WiFi, Fr...",A safe haven in the middle of the metropolis: ...,Rs3734.52,Rs20752.88
1,St Christopher's Village,- 1.8km from city centre,8.2,[12390],Fabulous,"[Linen Included, Free WiFi, Free Internet Acce...",Hostelworld visitors have said St Christopher'...,-,Rs2843.70
2,Onefam Notting Hill,- 5.5km from city centre,9.7,[2233],Superb,"[Linen Included, Free WiFi, Free Internet Acce...",The perfect place for solo travelers to connec...,Non-refundable,Non-refundable
3,NX London Hostel,- 6.1km from city centre,8.3,[2095],Fabulous,"[Free Breakfast, Linen Included, Towels Includ...",Welcome to NX London Hostel!\n\nPLEASE READ IN...,-,-
4,Urbany Hostel London,- 5.4km from city centre,9.5,[895],Superb,"[Linen Included, Free WiFi, Security Lockers, ...","Welcome to Urbany Hostel London, our first int...",-,Non-refundable
...,...,...,...,...,...,...,...,...,...
61,The Dover,- 1.9km from city centre,No Rating\n0 Total Reviews,[0],-,"[Free Breakfast, Free City Maps, Towels Includ...","With friendly staff, excellent service and an ...",-,-
62,Wellesley Hotel,- 14.7km from city centre,No Rating\n0 Total Reviews,[0],-,"[Linen Included, Towels Included, Free WiFi, F...","In the Heart of Ilford, Greater London region,...",-,-
63,Budget London Apartments,- 8.2km from city centre,4.7,[366],Rating,"[Linen Included, Free WiFi, Free Internet Acce...",We offer the best budget accommodation located...,-,-
64,The London Home Hostel,- 8.2km from city centre,4.9,[268],Rating,"[Linen Included, Free WiFi, Free Internet Acce...",If you are looking to meet and socialise with ...,-,-
